In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
import numpy as np

from Newmark import *
from meshes import *

from get_M import *
from get_S import *

from NumericalSolutions import *

In [ ]:
# Spatial resolution of mesh
N = 25

# Right limit of mesh
L = 1

# Generate 1D mesh
[nodes, elems, faces] = get_mesh_1D(N, [0,L], True);
elems = np.array(elems)[:-1,:]          # I believe there is one row too much in elems from get_mesh function

In [ ]:
E=1
I=1
mu = 1

In [ ]:
M0 = 0
ML = 0
QL = 0
a0 = 0
aL = 0
a = 0
b = 0

q = 1

two_sided_support = True 

if two_sided_support:
    boundary_conditions = {'M0': M0, 'ML': ML, 'a0': a0, 'aL': aL}
else:
    boundary_conditions = {'QL': QL, 'ML': ML, 'a': a, 'b': b}

In [ ]:
DynamicCase = Dynamic(E=E, I=I, N=N, L=L, 
                      initial_conditions = {"ML":ML, "M0":M0, "a0":a0, "aL":aL, "q":0},
                      #parameters = {"ML":ML, "M0":M0, "a0":a0, "aL":aL, "q":4},
                      parameters = {"ML":ML, "M0":M0, "a0":0, "aL":5, "q":4},
                      two_sided_support=True)

tsteps = 100

DynamicCase.Newmark.h = 0.05
DynamicCase.Newmark.run(tsteps)

In [ ]:
# get animations

%matplotlib inline

from matplotlib import animation, rc
from IPython.display import HTML


results = DynamicCase.Newmark.results
w = results["w"]

wmin = np.min(w)
wmax = np.max(w)

fig, ax = plt.subplots(figsize=(10,10))

ax.set_xlim(( 0, L))
ax.set_ylim((wmin - 0.05 , wmax + 0.05))
#try: ax.set_ylim((np.min(-u[nsteps-1][:-2][::2])-0.5, np.max(u[nsteps-1][:-2][::2])+0.5))
#except: ax.set_ylim((np.min(-solution), np.max(solution)))
        
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(nodes, w[i])
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=tsteps, interval=50, 
                               blit=True)
HTML(anim.to_jshtml())